## Base Case with Multiple Stock - Duality

In this nb, we solve the cutting optima problem with big order quantity to cut many specs at the same time with many stocks ( stock with same width and nearly same weight)
The condition/constraints as below: 
- `Generate Pattern` that fit in stock choice and sum each FG cut > lower bound demand
- `Filter Pattern` that keep the trim_cost <= 4%
- `Cut Pattern` that cut many stocks with filtered_pattern to minimize the stocks used

### 0. Helpers

In [19]:
SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

In [20]:
from typing import Dict, Any

def check_finish_weight_per_stock(weight_s: float, width_s: float, finish: Dict[str, Dict[str, Any]]) -> Dict[str, bool]:
    """
    DOMAIN KNOWLEDGE:
    - As upper_demand is slightly/or significant higher than need_cut, 
    - We can check to pick stock that have weight per slice cut < upper_demand
    -  So that, at least, we can cut 1 slice of the FG with that kind of stock

    PICK STOCK WITH ALL/ALMOST TRUE VALUE (SUM TRUE LARGEST)
    """
    # MC WEIGHT PER UNIT
    wu = weight_s / width_s
    # dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}
    # dict of upperbound demand FG on the chosen stock
    f_upper_demand = {f: finish[f]["upper_bound"] for f in finish.keys()}

    # dict of Yes/No 
    check_f ={f: weight_slice_f[f] < f_upper_demand[f] for f in finish.keys()}
    num_var_f = {f: 1 if weight_slice_f[f] < f_upper_demand[f] else 0 for f in finish.keys()}

    return check_f, num_var_f

def make_naive_patterns(stocks, finish, MIN_MARGIN):
    """
    Generates patterns of feasible cuts from stock width to meet specified finish widths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
                   
                   Naive pattern with maximum number of cuts of each Finished Goods
                   that is closet to the required need_cut
                   and SUM(widths of FG) smaller Mother Coil width
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts_by_width = int((stocks[s]["width"]-MIN_MARGIN) / finish[f]["width"])
            # max number of f that satisfied the need cut WEIGHT BOUND
            num_cuts_by_weight = int((finish[f]["upper_bound"] * stocks[s]["width"] ) / (finish[f]["width"] * stocks[s]['weight']))
            # min of two max will satisfies both
            num_cuts = min(num_cuts_by_width, num_cuts_by_weight)

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for Stock {s} and FG {f}")

    return patterns

In [21]:
def finish_demand_by_slice(stock, finish: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Convert demand in KGs to demand in slice on a specific stock
    """
    # MC WEIGHT PER UNIT
    wu = stock['weight'] / stock['width']
    # Dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}

    new_finish_list = {}
    for f, finish_info in finish.items():
         new_finish_list[f] = {**finish_info
                            ,"upper_demand_slice": int(finish[f]["upper_bound"]/weight_slice_f[f])
                            , "demand_slice": int(finish[f]["need_cut"]/weight_slice_f[f])}
    
    # Filtering the dictionary to include only items with keys in keys_to_keep
    filtered_finish_list = {k: v for k, v in new_finish_list.items() if v['upper_demand_slice'] > 0}

    return filtered_finish_list


### 1. Loading data

In [22]:
## PARAMETER 
#   DOMAIN KNOWLEDGE:
# - User usually overcut current order_quantity (30 -50%) default on model
# - Users want to cut all PO qty in one stock cut ( increase boundary to max 3 months forecast)
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

MARGIN_DICT = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = MARGIN_DICT[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

## PARAMETER
BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_KEY:{BOUND_KEY}, BOUND_VALUE:{BOUND_VALUE}")
LIMIT_BOUND_KEY, LIMIT_BOUND_VALUE =  "limited", None

MIN_MARGIN:6
BOUND_KEY:limited, BOUND_VALUE:None


In [23]:
# TEST DATA - NORMAL CASE , NO DEFECT, UPPER BOUND DEMAND IS 50% PO QTY (TRY THIS CASE FIRST)
stocks = {
    "S0": {"width": 236, "weight": 1500, "qty": 1},
    "S1": {"width": 1219, "weight": 4395, "qty": 5 },
    "S2": {"width": 1219, "weight": 5260, "qty": 3},
    "S3": {"width": 1018, "weight": 3475, "qty": 3},
    "S4": {"width": 1219, "weight": 8535, "qty": 2},
    # "S5": {"width": 236, "weight": 1571, "qty": 1},
    "S5": {"width": 1219, "weight": 9260, "qty": 3},
}

finish = {
    "F1": {"width": 235, "need_cut": 800, "upper_bound": 1040},
    "F2": {"width": 147, "need_cut": 3308, "upper_bound": 3700},
    "F3": {"width": 136, "need_cut": 2290, "upper_bound": 3677},
    "F4": {"width": 68, "need_cut": 1600, "upper_bound": 1800},
    "F5": {"width": 60, "need_cut": 270, "upper_bound": 521},
    "F6": {"width": 85, "need_cut": 132, "upper_bound": 171},
    "F7": {"width": 57, "need_cut": 100, "upper_bound": 130},
    "F8": {"width": 92, "need_cut": 100, "upper_bound": 130}, 
    "F9": {"width": 57, "need_cut": 735, "upper_bound": 955},
}

# Test if SUM(need_cut) >> largest stock (in weight) -> Multi-stock Cut ( Can pick stock with available number, or stock with nearly same weight - same width)
# If FG have small spec and SUM(need_cut) can fit into s stock -> Single-stock Cut

### 2. Filter with list of Stocks (highest qty) can fit as-many-as possible FG

In [24]:
# DOMAIN KNOWLEDGE:
# - FG with small spec - weight is hard to fit in a large weight stock

def filter_high_fit_stock(stocks,finish):
    new_stock_list = {}
    max_f_fit = 0
    for stock_name, stock_info in stocks.items():
        _, num_var_f = check_finish_weight_per_stock(stock_info["weight"], 
                                                     stock_info["width"], finish)
        total_f_fit = sum(num_var_f.values())

        if total_f_fit > max_f_fit:
            #new_stock_list is reset 
            new_stock_list = {stock_name: {**stock_info, "total_f_fit": total_f_fit}} 
            max_f_fit = total_f_fit
        elif total_f_fit == max_f_fit:
            new_stock_list[stock_name] = {**stock_info, "total_f_fit": total_f_fit}
    
    return new_stock_list

high_stock_list = filter_high_fit_stock(stocks,finish)
high_stock_list

{'S1': {'width': 1219, 'weight': 4395, 'qty': 5, 'total_f_fit': 6},
 'S2': {'width': 1219, 'weight': 5260, 'qty': 3, 'total_f_fit': 6},
 'S3': {'width': 1018, 'weight': 3475, 'qty': 3, 'total_f_fit': 6}}

### 3. Solve with duality for pattern generation

In [75]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

# NEW PATTERN
def new_pattern_problem(finish, width_s, ap_upper_bound, demand_duals,MIN_MARGIN):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cost:
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints
    # subject to stock_length:
    #    sum{f in F} ap[f] * length_f[f] <= length_s;
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s - MIN_MARGIN, "StockLength"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY
def generate_pattern_dual(stocks, finish, patterns, MIN_MARGIN):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    # c = {p: stocks[s[p]]["cost"] for p in range(len(patterns))}
    
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand_slice"] for f in F}

    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    # ap_upper_bound = {
    #     f: max([int(stocks[s]["width"] / finish[f]["width"]) for s in stocks.keys()])
    #     for f in F
    # }
    ap_upper_bound = {f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)]) for f in finish.keys()}
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals,MIN_MARGIN
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

In [76]:
# CUT KNOWN SLICE PATTERNS
def cut_slice_patterns(finish, patterns):
    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(x[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * x[p] for p in range(len(patterns))) >= finish[f]['demand_slice'], f"DemandSlice{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(solution[p] for p in range(len(patterns)))

    return solution, total_cost

### 4. SOLVING

### >> Trial: Cut with a chosen stock
### >>> S1

In [83]:
## Pick Stock
s = "S1"
choosen_stock_list = {k: v for k, v in high_stock_list.items() if k==s}
print(f"Choose Stock: {choosen_stock_list}")

# Pick Finish List
filtered_finish_list = finish_demand_by_slice(stocks[s], finish)
sum_needcut = sum(item['need_cut'] for item in filtered_finish_list.values())
print(f"Sum need cut:{sum_needcut}")   

# Create Naive Patterns
patterns = make_naive_patterns(choosen_stock_list, filtered_finish_list, MIN_MARGIN)
print(f"number patterns: {len(patterns)}")

Choose Stock: {'S1': {'width': 1219, 'weight': 4395, 'qty': 5, 'total_f_fit': 6}}
Sum need cut:9003
number patterns: 6


In [85]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'len sum patterns :{len(patterns)}')
print()
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []
width_s = choosen_stock_list[s]["width"] 
for pattern in patterns:
    cuts_dict= pattern['cuts']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in filtered_finish_list.keys()])
    trim_loss_pct = trim_loss/width_s*100
    if trim_loss_pct <= 4.00:
        print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        print(f"trimloss: {trim_loss},trim loss percent:{trim_loss_pct}") 

Phase 1: Pattern Generation.
len sum patterns :14

Phase 2: Filter Patterns:
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 3, 'F3': 0, 'F4': 6, 'F5': 2, 'F9': 4}} *****
trimloss: 22,trim loss percent:1.8047579983593112
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 5, 'F3': 0, 'F4': 7, 'F5': 0, 'F9': 0}} *****
trimloss: 8,trim loss percent:0.6562756357670222
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 3, 'F3': 4, 'F4': 0, 'F5': 0, 'F9': 4}} *****
trimloss: 6,trim loss percent:0.4922067268252666
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 0, 'F3': 7, 'F4': 2, 'F5': 2, 'F9': 0}} *****
trimloss: 11,trim loss percent:0.9023789991796556
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 6, 'F3': 1, 'F4': 0, 'F5': 2, 'F9': 1}} *****
trimloss: 24,trim loss percent:1.9688269073010665
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 4, 'F3': 0, 'F4': 4, 'F5': 2, 'F9': 4}} *****
trimloss: 11,trim loss percent:0.902378999179

In [86]:
# Phrase 3: Cut Pattern to map all pattern to demand requirements
print("Phase 3: Cut Patterns", end=".")
print()
solution, total_cost = cut_slice_patterns(filtered_finish_list, filtered_trimloss_pattern)
for i, s in enumerate(solution):
    if s > 0:
        print(f"Take ({int(s)}) pattern: {filtered_trimloss_pattern[i]['cuts']}")

Phase 3: Cut Patterns.
Take (1) pattern: {'F1': 0, 'F2': 3, 'F3': 0, 'F4': 6, 'F5': 2, 'F9': 4}
Take (1) pattern: {'F1': 0, 'F2': 3, 'F3': 4, 'F4': 0, 'F5': 0, 'F9': 4}


#### >>> S2

In [106]:
## Pick Stock
s = "S2"
choosen_stock_list = {k: v for k, v in high_stock_list.items() if k==s}
print(f"Choose Stock: {choosen_stock_list}")

# Pick Finish List
filtered_finish_list = finish_demand_by_slice(stocks[s],finish)
sum_needcut = sum(item['need_cut'] for item in filtered_finish_list.values())
print(f"Sum need cut:{sum_needcut}")

# Create Naive Patterns
patterns = make_naive_patterns(choosen_stock_list, filtered_finish_list, MIN_MARGIN)
print(f"number patterns: {len(patterns)}")

Choose Stock: {'S2': {'width': 1219, 'weight': 5260, 'qty': 3, 'total_f_fit': 6}}
Sum need cut:9003
number patterns: 6


In [107]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'len sum patterns :{len(patterns)}')
print()
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []
width_s = choosen_stock_list[s]["width"] 
for pattern in patterns:
    cuts_dict= pattern['cuts']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in filtered_finish_list.keys()])
    trim_loss_pct = trim_loss/width_s*100
    if trim_loss_pct <= 4.00:
        print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        print(f"trimloss: {trim_loss},trim loss percent:{trim_loss_pct}") 

Phase 1: Pattern Generation............
len sum patterns :17

Phase 2: Filter Patterns:
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 5, 'F3': 3, 'F4': 1, 'F5': 0, 'F9': 0}} *****
trimloss: 8,trim loss percent:0.6562756357670222
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 2, 'F3': 3, 'F4': 5, 'F5': 0, 'F9': 3}} *****
trimloss: 6,trim loss percent:0.4922067268252666
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 5, 'F3': 0, 'F4': 6, 'F5': 1, 'F9': 0}} *****
trimloss: 16,trim loss percent:1.3125512715340444
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 1, 'F3': 6, 'F4': 1, 'F5': 2, 'F9': 1}} *****
trimloss: 11,trim loss percent:0.9023789991796556
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 3, 'F3': 3, 'F4': 1, 'F5': 2, 'F9': 3}} *****
trimloss: 11,trim loss percent:0.9023789991796556
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 2, 'F3': 2, 'F4': 6, 'F5': 2, 'F9': 2}} *****
trimloss: 11,trim loss percent:0.9

In [108]:
# Phrase 3: Cut Pattern to map all pattern to demand requirements
print("Phase 3: Cut Patterns", end=".")
print()
solution, total_cost = cut_slice_patterns(filtered_finish_list, filtered_trimloss_pattern)
for i, s in enumerate(solution):
    if s > 0:
        print(f"Take ({int(s)}) pattern: {filtered_trimloss_pattern[i]['cuts']}")

Phase 3: Cut Patterns.
Take (1) pattern: {'F1': 0, 'F2': 5, 'F3': 3, 'F4': 1, 'F5': 0, 'F9': 0}
Take (1) pattern: {'F1': 0, 'F2': 3, 'F3': 1, 'F4': 5, 'F5': 2, 'F9': 3}


#### >>> S3

In [109]:
## Pick Stock
s = "S3"
choosen_stock_list = {k: v for k, v in high_stock_list.items() if k==s}
print(f"Choose Stock: {choosen_stock_list}")

# Pick Finish List
filtered_finish_list = finish_demand_by_slice(stocks[s],finish)
sum_needcut = sum(item['need_cut'] for item in filtered_finish_list.values())
print(f"Sum need cut:{sum_needcut}")

# Create Naive Patterns
patterns = make_naive_patterns(choosen_stock_list, filtered_finish_list, MIN_MARGIN)
print(f"number patterns: {len(patterns)}")

Choose Stock: {'S3': {'width': 1018, 'weight': 3475, 'qty': 3, 'total_f_fit': 6}}
Sum need cut:9003
number patterns: 6


In [110]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(choosen_stock_list, filtered_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'len sum patterns :{len(patterns)}')
print()
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []
width_s = choosen_stock_list[s]["width"] 
for pattern in patterns:
    cuts_dict= pattern['cuts']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in filtered_finish_list.keys()])
    trim_loss_pct = trim_loss/width_s * 100
    if trim_loss_pct <= 4.00:
        print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        print(f"trim loss: {trim_loss},trim loss percent:{trim_loss_pct}") 

Phase 1: Pattern Generation...........
len sum patterns :16

Phase 2: Filter Patterns:
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 4, 'F3': 0, 'F4': 6, 'F5': 0, 'F9': 0}} *****
trimloss: 22,trim loss percent:2.161100196463654
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 3, 'F3': 2, 'F4': 1, 'F5': 0, 'F9': 4}} *****
trimloss: 9,trim loss percent:0.8840864440078585
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 6, 'F3': 0, 'F4': 0, 'F5': 2, 'F9': 0}} *****
trimloss: 16,trim loss percent:1.5717092337917484
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 0, 'F3': 3, 'F4': 7, 'F5': 2, 'F9': 0}} *****
trimloss: 14,trim loss percent:1.37524557956778
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 0, 'F3': 7, 'F4': 0, 'F5': 1, 'F9': 0}} *****
trimloss: 6,trim loss percent:0.5893909626719057
***** take pattern {'stock': 'S3', 'cuts': {'F1': 0, 'F2': 5, 'F3': 0, 'F4': 4, 'F5': 0, 'F9': 0}} *****
trimloss: 11,trim loss percent:1.08055

In [111]:
# Phrase 3: Cut Pattern to map all pattern to demand requirements
print("Phase 3: Cut Patterns", end=".")
print()
solution, total_cost = cut_slice_patterns(filtered_finish_list, filtered_trimloss_pattern)
for i, s in enumerate(solution):
    if s > 0:
        print(f"Take ({int(s)}) pattern: {filtered_trimloss_pattern[i]['cuts']}")

Phase 3: Cut Patterns.
Take (1) pattern: {'F1': 0, 'F2': 3, 'F3': 2, 'F4': 1, 'F5': 0, 'F9': 4}
Take (1) pattern: {'F1': 0, 'F2': 6, 'F3': 0, 'F4': 0, 'F5': 2, 'F9': 0}
Take (1) pattern: {'F1': 0, 'F2': 1, 'F3': 2, 'F4': 7, 'F5': 1, 'F9': 1}


### >> Create Pattern with all stocks

In [112]:
# DUALITY
def generate_pattern_dual_w(stocks, finish, patterns, MIN_MARGIN):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    c = {p: stocks[s[p]]["weight"]/stocks[s[p]]["weight"] for p in range(len(patterns))}
    
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["need_cut"] for f in F}

    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * x[p] * c[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)]) for f in finish.keys()}
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals,MIN_MARGIN
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

In [113]:
patterns = make_naive_patterns(high_stock_list, finish, MIN_MARGIN)
print(f"number patterns: {len(patterns)}")

No feasible pattern was found for Stock S3 and FG F6
No feasible pattern was found for Stock S3 and FG F7
No feasible pattern was found for Stock S3 and FG F8
number patterns: 18


In [114]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual_w(high_stock_list, finish, patterns, MIN_MARGIN)

while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual_w(high_stock_list, finish, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'len sum patterns : {len(patterns)}')
print()

Phase 1: Pattern Generation.......
len sum patterns :24



In [115]:
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
unique_stocks = {pattern['stock'] for pattern in patterns}
filtered_trimloss_pattern = []
for s in unique_stocks:
    width_s = high_stock_list[s]["width"] 
    for pattern in patterns:
        cuts_dict= pattern['cuts']
        trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in filtered_finish_list.keys()])
        trim_loss_pct = trim_loss/width_s*100
        if 0 < trim_loss_pct <= 4.00:
            print(f"***** take pattern {pattern} *****")
            filtered_trimloss_pattern.append(pattern)
            print(f"trimloss: {trim_loss},trim loss percent:{trim_loss_pct}") 

Phase 2: Filter Patterns:
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 1, 'F3': 0, 'F4': 7, 'F5': 2, 'F6': 0, 'F7': 0, 'F8': 0, 'F9': 4}} *****
trimloss: 13,trim loss percent:1.0664479081214109
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 2, 'F3': 5, 'F4': 0, 'F5': 0, 'F6': 0, 'F7': 0, 'F8': 0, 'F9': 0}} *****
trimloss: 10,trim loss percent:0.8203445447087777
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 6, 'F3': 0, 'F4': 1, 'F5': 0, 'F6': 0, 'F7': 0, 'F8': 0, 'F9': 0}} *****
trimloss: 34,trim loss percent:2.7891714520098443
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 1, 'F3': 0, 'F4': 7, 'F5': 2, 'F6': 0, 'F7': 0, 'F8': 0, 'F9': 4}} *****
trimloss: 13,trim loss percent:1.0664479081214109
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 2, 'F3': 5, 'F4': 0, 'F5': 0, 'F6': 0, 'F7': 0, 'F8': 0, 'F9': 0}} *****
trimloss: 10,trim loss percent:0.8203445447087777
***** take pattern {'stock': 'S1', 'cuts': {'F1': 1, 'F2': 6, 'F

In [ ]:
# CUT KNOWN WEIGHT PATTERNS
def cut_weight_patterns(stocks, finish, patterns):
    # Parameters
    c = {p: stocks[s[p]]["weight"]/stocks[s[p]]["width"] for p in range(len(patterns))}

    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(x[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * finish[f]['width'] * x[p] for p in range(len(patterns))) >= finish[f]['need_cut'], f"DemandWeight{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(1 * solution[p] for p in range(len(patterns)))

    return solution, total_cost